In [1]:
import re
from google_drive_downloader import GoogleDriveDownloader as gdd

In [5]:
gdd.download_file_from_google_drive(file_id="1moAgo9pulaAMUCTvcqjdNAefyL-I35o0",
                                    dest_path="./train_data.txt",
                                    unzip=False)

In [6]:
with open('train_data.txt', 'r', encoding='utf-8') as f:
    text = f.read()
    text = text.replace('\ufeff', '') # убираем первый невидимый символ
    text = re.sub(r'[^А-я ]', '', text) # убираем все недопустимые символы

In [7]:
text

'Вы  лучший ответ на проблемы которые возникли в понедельникДумайте позитивно и верьте в свою способность достигать отличных результатовЕсли вы смогли в понедельник подняться с постели значит вы супер герой'

In [8]:
len(text)

205

In [27]:
num_characters = len(set(text)) #уникальные символы
num_characters                  #len(np.unique(np.array([x for x in text])))

30

In [28]:
import keras
from keras.preprocessing.text import Tokenizer

In [29]:
tokenizer = Tokenizer(num_words=num_characters, char_level=True)

In [30]:
tokenizer.fit_on_texts(text)

In [31]:
print(tokenizer.word_index)

{' ': 1, 'о': 2, 'т': 3, 'е': 4, 'и': 5, 'в': 6, 'н': 7, 'с': 8, 'л': 9, 'п': 10, 'ь': 11, 'ы': 12, 'р': 13, 'а': 14, 'д': 15, 'у': 16, 'к': 17, 'з': 18, 'ч': 19, 'й': 20, 'м': 21, 'г': 22, 'б': 23, 'я': 24, 'ш': 25, 'ю': 26, 'х': 27}


In [100]:
inp_chars = 3 #кол-во вводимых символов
data = tokenizer.texts_to_matrix(text)

In [33]:
data[1]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [101]:
n = data.shape[0]-inp_chars
n  #размер обучающего множества

202

In [42]:
import numpy as np

In [102]:
# генерация обучающего набора
X = np.array([data[i:i+inp_chars, :] for i in range(n)])
Y = data[inp_chars:] #предсказание следующего символа

In [103]:
len(X), len(Y)

(202, 202)

In [51]:
X[0]

array([[0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [52]:
Y[0]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [53]:
from keras.layers import *
from keras.models import Sequential

In [105]:
model = Sequential()
model.add(Input((inp_chars, num_characters)))
model.add(SimpleRNN(500, activation='tanh'))
model.add(Dense(num_characters, activation='softmax'))
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn_2 (SimpleRNN)    (None, 500)               265500    
                                                                 
 dense_2 (Dense)             (None, 30)                15030     
                                                                 
Total params: 280530 (1.07 MB)
Trainable params: 280530 (1.07 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [107]:
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')
history = model.fit(X, Y, batch_size=32, epochs=100)

Epoch 1/100
7/7 [==============================] - 1s 11ms/step - loss: 0.7406 - accuracy: 0.7723
Epoch 2/100
7/7 [==============================] - 0s 10ms/step - loss: 0.6499 - accuracy: 0.7673
Epoch 3/100
7/7 [==============================] - 0s 11ms/step - loss: 0.6195 - accuracy: 0.8168
Epoch 4/100
7/7 [==============================] - 0s 11ms/step - loss: 0.5228 - accuracy: 0.8267
Epoch 5/100
7/7 [==============================] - 0s 10ms/step - loss: 0.5031 - accuracy: 0.8366
Epoch 6/100
7/7 [==============================] - 0s 11ms/step - loss: 0.4705 - accuracy: 0.8317
Epoch 7/100
7/7 [==============================] - 0s 11ms/step - loss: 0.4500 - accuracy: 0.8416
Epoch 8/100
7/7 [==============================] - 0s 10ms/step - loss: 0.4345 - accuracy: 0.8663
Epoch 9/100
7/7 [==============================] - 0s 13ms/step - loss: 0.4551 - accuracy: 0.8465
Epoch 10/100
7/7 [==============================] - 0s 12ms/step - loss: 0.4335 - accuracy: 0.8416
Epoch 11/100
7/7 [=

In [218]:
def buildPhrase(inp_str, str_len = 1):
  _inp_chars = inp_chars if len(inp_str) > inp_chars else len(inp_str)

  for i in range(str_len):
    x = []
    for j in range(len(inp_str) - _inp_chars, len(inp_str)):
      x.append(tokenizer.texts_to_matrix(inp_str[j])) # преобразуем символы в One-Hot-encoding

    x = np.array(x)

    inp = x.reshape(1, inp_chars, num_characters)

    pred = model.predict( inp ) # предсказываем OHE четвертого символа
    char = tokenizer.index_word[pred.argmax(axis=1)[0]] # получаем ответ в символьном представлении

    inp_str += char # дописываем строку

  return inp_str

In [227]:
print(buildPhrase("позитивн", 7))
print(buildPhrase("способ", 15))
print(buildPhrase("проблемы", 21))

1/1 [==============================] - 0s 24ms/step
позитивно и вы 
1/1 [==============================] - 0s 73ms/step
способность отличных 
1/1 [==============================] - 0s 23ms/step
проблемы которые в пособность


# Слова


In [ ]:
with open('train_data.txt', 'r', encoding='utf-8') as f:
    texts = f.read()
    texts = texts.replace('\ufeff', '') # убираем первый невидимый символ

In [ ]:
maxWordsCount = 1000
tokenizer = Tokenizer(num_words=maxWordsCount, filters='!–"—#$%&amp;()*+,-./:;<=>?@[\\]^_`{|}~\t\n\r«»',
                       lower=True, split=' ', char_level=False)
tokenizer.fit_on_texts([texts])

In [ ]:
dist = list(tokenizer.word_counts.items())
print(dist[:10])

[('вы', 3), ('лучший', 1), ('ответ', 1), ('на', 1), ('проблемы', 1), ('которые', 1), ('возникли', 1), ('в', 3), ('понедельник', 2), ('думайте', 1)]


In [ ]:
data = tokenizer.texts_to_sequences([texts])

In [ ]:
data

[[1,
  4,
  5,
  6,
  7,
  8,
  9,
  2,
  3,
  10,
  11,
  12,
  13,
  2,
  14,
  15,
  16,
  17,
  18,
  19,
  1,
  20,
  2,
  3,
  21,
  22,
  23,
  24,
  1,
  25,
  26]]

In [ ]:
len(data[0])

31

In [ ]:
res = keras.utils.to_categorical(data[0], num_classes=maxWordsCount)
print( res.shape )

(31, 1000)


In [ ]:
inp_words = 3
n = res.shape[0]-inp_words

In [ ]:
X = np.array([res[i:i+inp_words, :] for i in range(n)])
Y = res[inp_words:]

In [ ]:
model = Sequential()
model.add(Input((inp_words, maxWordsCount)))
model.add(SimpleRNN(128, activation='tanh'))
model.add(Dense(maxWordsCount, activation='softmax'))
model.summary()

model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn_4 (SimpleRNN)    (None, 128)               144512    
                                                                 
 dense_4 (Dense)             (None, 1000)              129000    
                                                                 
Total params: 273512 (1.04 MB)
Trainable params: 273512 (1.04 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
history = model.fit(X, Y, batch_size=32, epochs=19)

Epoch 1/19
1/1 [==============================] - 1s 1s/step - loss: 6.9144 - accuracy: 0.0000e+00
Epoch 2/19
1/1 [==============================] - 0s 16ms/step - loss: 6.8882 - accuracy: 0.0000e+00
Epoch 3/19
1/1 [==============================] - 0s 25ms/step - loss: 6.8618 - accuracy: 0.0357
Epoch 4/19
1/1 [==============================] - 0s 16ms/step - loss: 6.8351 - accuracy: 0.1071
Epoch 5/19
1/1 [==============================] - 0s 18ms/step - loss: 6.8077 - accuracy: 0.2857
Epoch 6/19
1/1 [==============================] - 0s 22ms/step - loss: 6.7793 - accuracy: 0.5714
Epoch 7/19
1/1 [==============================] - 0s 16ms/step - loss: 6.7497 - accuracy: 0.7857
Epoch 8/19
1/1 [==============================] - 0s 21ms/step - loss: 6.7185 - accuracy: 0.9286
Epoch 9/19
1/1 [==============================] - 0s 15ms/step - loss: 6.6855 - accuracy: 0.9643
Epoch 10/19
1/1 [==============================] - 0s 17ms/step - loss: 6.6502 - accuracy: 0.9643
Epoch 11/19
1/1 [======

In [ ]:
def buildPhrase(texts, str_len = 20):
  result = texts
  data = tokenizer.texts_to_sequences([texts])[0]
  for i in range(len(data)):
    x = keras.utils.to_categorical(data[i: i+inp_words], num_classes=maxWordsCount) # преобразуем в One-Hot-encoding
    inp = x.reshape(1, inp_words, maxWordsCount)
    pred = model.predict( inp ) # предсказываем OHE четвертого символа
    indx = pred.argmax(axis=1)[0]
    data.append(indx)
    result += " " + tokenizer.index_word[indx] # дописываем строку

  return result

In [ ]:
for i in range(1):
    x = keras.utils.to_categorical(data[i: i+inp_words], num_classes=maxWordsCount)

In [ ]:
x.shape

(1, 31, 1000)

In [ ]:
res = buildPhrase("значит вы")
print(res)

ValueError: cannot reshape array of size 2000 into shape (1,3,1000)